# Practico 5 (parte 4)

## Entrenar word embeddings

## Importación de módulos y librerías

In [1]:
# Inclusion de librerias y módulos
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import nltk

# Algunas utilidades
from utiles import print_some_info

# Nos permite convertir str a list
from ast import literal_eval

# Colores
BLUE   = '#5DADE2'
RED    = '#ff7043'
ORANGE = '#F5B041'
GREEN  = '#58D68D'
YELLOW = '#F4D03F'
pltcolors = [BLUE, RED, ORANGE, GREEN, YELLOW]

# Plot axes y legends parambs
plt.rcParams["axes.labelweight"]   = "bold"
plt.rcParams["axes.titleweight"]   = "bold"
plt.rcParams["legend.shadow"]      = True
plt.rcParams["figure.titleweight"] = "bold"

data_dir = os.path.join('..', 'dataset')

## Lectura del archivo de mensajes
Utilizamos unicamente el archivo de mensajes dado que vamos a entrenar un word embeding como word2vec. Entendemos que para el propósito del análisis y por que no estamos empleando ningún modelo de clasificación o regresión podemos usar el conjunto de datos completo.

In [2]:
filename = 'dev_yup_messages_preprocessed.csv'
df = pd.read_csv(os.path.join(data_dir, filename))

print(f'El conjunto de datos utilizado es {filename}')
print_some_info(df)


El conjunto de datos utilizado es dev_yup_messages_preprocessed.csv
El conjunto de datos posee 234375 filas y 6 columnas
&lt;class &#39;pandas.core.frame.DataFrame&#39;&gt;
RangeIndex: 234375 entries, 0 to 234374
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   session_id    234375 non-null  int64 
 1   created_at    234375 non-null  object
 2   sent_from     234375 non-null  object
 3   sent_to       234375 non-null  object
 4   content_type  234375 non-null  object
 5   text          234375 non-null  object
dtypes: int64(1), object(5)
memory usage: 10.7+ MB
None


In [33]:
import re
from emot.emo_unicode import UNICODE_EMO, EMOTICONS
def convert_emojis(text):
    text0 = [t for t in re.findall(r'\\x..\\x..\\x..\\x..', str(text.encode()))]
    text0 = [''.join(t.split('\\x')[1:]) for t in text0]
    text0 = [bytes.fromhex(t).decode() for t in text0 if t[0]=='f']
    text0 = [UNICODE_EMO[t] for t in text0 if t in UNICODE_EMO]
    text0 = ' '.join(text0)
    return text0 if text0 else text

In [6]:
#1. Tomamos solo las columnas que nos pueden servir. Esto es preliminar, podríamos tomar solo `text`
dfclean = df[['session_id', 'sent_from', 'text']]

#2. Tomamos solo las filas que sean tutor o student a partir de la columna `sent_from`
dfclean = dfclean[dfclean.sent_from.isin(['student', 'tutor'])]

#3. Convertimos a lista de strings el contenido de la columna text
dfclean['text'] = dfclean.text.apply(lambda x: literal_eval(x))

dfclean

# 4 emojis

# 5 emoticones

# 6 minusculas

# 7 stopwords

,session_id,sent_from,text
0,299895,tutor,"[Welcome, to, Yup, ,, Jasmyn, !]"
1,299895,tutor,"[How, may, I, help, you, with, your, math, tod..."
3,299899,student,[<url>]
5,299899,tutor,"[Hello, Luke, ,, welcome, to, Yup, !]"
6,299899,tutor,"[I, am, looking, over, your, problem, .]"
...,...,...,...
234370,326164,student,"[And, then, I, distribute, what, s, in, the, p..."
234371,326164,tutor,"[Yep, !]"
234372,326164,student,"[I, m, almost, done]"
234373,326164,tutor,"[Okay, !, Send, me, a, pic, of, your, work, wh..."


In [36]:
dfclean.text.sample(20)


229384                               [I, got, 0.6986768577]
213374                                    [I, &#39;m, confused]
145736           [Ok, ., Can, you, send, me, the, graph, ?]
231519          [Remember, 400, workers, take, 8, years, .]
205763         [Awesome, :), You, got, that, one, quick, !]
194814                                  [No, thank, you, !]
121124                                              [which]
201946                                              [27/38]
151405                                  [Is, this, correct]
38630                                               [&lt;url&gt;]
176333                                [but, what, about, y]
60441     [If, ONE, box, A, contains, 100, g, of, COCO, ...
9683                                         [I, mean, yes]
166701                                      [Well, done, !]
152863                                              [&lt;url&gt;]
69848                      [so, how, do, i, write, that, ?]
160616                  

In [35]:
[convert_emojis(i) for i in dfclean.loc[148142].text]

[&#39;Not&#39;,
 &#39;much&#39;,
 &#39;at&#39;,
 &#39;all&#39;,
 &#39;I&#39;,
 &#39;did&#39;,
 &quot;n&#39;t&quot;,
 &#39;know&#39;,
 &#39;how&#39;,
 &#39;to&#39;,
 &#39;do&#39;,
 &#39;this&#39;,
 &#39;problem&#39;,
 &#39;so&#39;,
 &#39;I&#39;,
 &#39;need&#39;,
 &#39;ur&#39;,
 &#39;help&#39;]